In [6]:
"""
Notebook: Análise de Dados Imobiliários

Este notebook lê um conjunto de dados local, verifica/cria o banco MySQL,
cria tabelas para cada CSV e insere os dados no banco de forma automatizada.
"""

import os
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Configurações do banco de dados
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "12344",
    "database": "mercado_imobiliario"
}

# Caminho da pasta de dados
extract_path = "data"

# Função para conectar ao banco de dados
def conectar_banco():
    try:
        conexao = mysql.connector.connect(**DB_CONFIG)
        return conexao
    except Error as e:
        print(f"Erro ao conectar ao banco: {e}")
        return None

# Função para criar tabela no banco de dados para cada arquivo CSV
def criar_tabela(nome_tabela, df, conexao):
    try:
        cursor = conexao.cursor()

        # Ajustar os nomes das colunas para evitar caracteres inválidos
        df.columns = [col.replace(" ", "_").replace("-", "_") for col in df.columns]
        
        # Criar estrutura de colunas baseado nos tipos de dados
        colunas = []
        for col in df.columns:
            col_formatado = f"`{col}` TEXT"  # Definimos TEXT para evitar conflitos com tipos diferentes
            colunas.append(col_formatado)
        
        colunas_str = ", ".join(colunas)

        # Criar tabela se não existir
        query = f"""
        CREATE TABLE IF NOT EXISTS `{nome_tabela}` (
            {colunas_str}
        );
        """
        cursor.execute(query)
        conexao.commit()
        print(f"✅ Tabela `{nome_tabela}` criada/verificada com sucesso.")
    except Error as e:
        print(f"❌ Erro ao criar tabela {nome_tabela}: {e}")
    finally:
        cursor.close()

# Função para inserir dados na tabela
def inserir_dados(nome_tabela, df, conexao):
    try:
        cursor = conexao.cursor()
        
        # Ajustar colunas
        colunas = ", ".join([f"`{col}`" for col in df.columns])
        valores = ", ".join(["%s"] * len(df.columns))

        query = f"INSERT INTO `{nome_tabela}` ({colunas}) VALUES ({valores})"
        
        cursor.executemany(query, df.fillna("NULL").values.tolist())
        conexao.commit()
        print(f"✅ Dados inseridos na tabela `{nome_tabela}` com sucesso.")
    except Error as e:
        print(f"❌ Erro ao inserir dados na tabela `{nome_tabela}`: {e}")
    finally:
        cursor.close()

# Função para carregar os dados e inseri-los no banco
def carregar_dados():
    print(f"📂 Verificando diretório: {extract_path}")
    if not os.path.exists(extract_path):
        print("❌ Erro: Diretório de dados não encontrado. Verifique se os arquivos estão no local correto.")
        return
    
    arquivos = [f for f in os.listdir(extract_path) if f.endswith(".csv")]
    if arquivos:
        conexao = conectar_banco()
        if conexao:
            for arquivo in arquivos:
                caminho_arquivo = os.path.join(extract_path, arquivo)
                print(f"\n📥 Carregando arquivo: {caminho_arquivo}")
                
                df = pd.read_csv(caminho_arquivo)
                
                # Ajustar nome da tabela (removendo extensão .csv)
                nome_tabela = os.path.splitext(arquivo)[0]
                
                criar_tabela(nome_tabela, df, conexao)
                inserir_dados(nome_tabela, df, conexao)
            
            conexao.close()
    else:
        print("❌ Nenhum arquivo CSV encontrado na pasta.")

# Função para verificar a existência do banco de dados e criá-lo caso não exista
def verificar_criar_banco():
    try:
        print("🗄️ Verificando banco de dados...")
        conexao = mysql.connector.connect(
            host=DB_CONFIG["host"],
            user=DB_CONFIG["user"],
            password=DB_CONFIG["password"]
        )
        cursor = conexao.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS mercado_imobiliario;")
        print("✅ Banco de dados verificado/criado com sucesso.")
    except Error as e:
        print(f"❌ Erro ao conectar ao banco: {e}")
    finally:
        if conexao and conexao.is_connected():
            cursor.close()
            conexao.close()

# 🚀 Executando o fluxo de leitura de dados
print("🔄 Iniciando pipeline de leitura de dados...")
verificar_criar_banco()
carregar_dados()
print("✅ Pipeline concluído. Pronto para processamento posterior.")


🔄 Iniciando pipeline de leitura de dados...
🗄️ Verificando banco de dados...
✅ Banco de dados verificado/criado com sucesso.
📂 Verificando diretório: data

📥 Carregando arquivo: data\sample_submission.csv
✅ Tabela `sample_submission` criada/verificada com sucesso.
✅ Dados inseridos na tabela `sample_submission` com sucesso.

📥 Carregando arquivo: data\test.csv
✅ Tabela `test` criada/verificada com sucesso.
✅ Dados inseridos na tabela `test` com sucesso.

📥 Carregando arquivo: data\train.csv
✅ Tabela `train` criada/verificada com sucesso.
✅ Dados inseridos na tabela `train` com sucesso.
✅ Pipeline concluído. Pronto para processamento posterior.
